In [1]:
import pandas as pd
import os
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
load_dotenv()



False

In [2]:
folderFile = r"C:\Users\gabriel.sandres\OneDrive - Sicoob\Área de Trabalho\cod_fonte_registro\registro-chamados-codigo-fonte\planilha_registro.xlsx"

In [3]:
df = pd.read_excel(folderFile,dtype={'Documento do cooperado': str})


In [4]:
df.head()

,Documento do cooperado,Cooperativa,Protocolo PLAD,Categoria,Serviço,Protocolo Visão,Observação
0,05534899174,3179,WP12345678,Pix,Dúvida Técnica,SPI20250610006497,NaN
1,05534899174,3179,COB123456789,Cobrança Bancária,Dúvida Negocial,NaN,Teste teste teste
2,05534899174,3179,COB123456789,PIX,Dúvida Técnica,SPI20250610006498,NaN
3,05534899174,3179,COB123456789,Poupança,Dúvida Negocial,NaN,Automação para registro
4,05534899174,3179,COB123456789,PIX,Dúvida Técnica,NaN,NaN


In [5]:
# Bloco 1 – Acessar a URL do Visão 360 (com webdriver-manager)
 
options = Options()
options.add_argument("--start-maximized")  # Abre o navegador em tela cheia
 
# Inicializa o driver automaticamente com webdriver-manager
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
 
# Acessa o Visão 360
url = "https://portal.sisbr.coop.br/auth/realms/sisbr/protocol/openid-connect/auth?&scope=openid&client_id=visao360-sisbr&response_type=code&redirect_uri=https://portal.sisbr.coop.br/visao360/consult"
driver.get(url)
 
time.sleep(5)

In [6]:
# Captura login e senha do .env
#usuario = os.getenv("LOGIN_USERNAME")
#senha = os.getenv("LOGIN_PASSWORD")

login = 'gabriel.sandres'
senha = 'C@tolica61136'

 
# Espera e preenche o campo de login
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(login)
 
# Preenche o campo de senha
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys(senha)
 
# Clica no botão "Logar"
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "kc-login"))).click()
 
# Espera o QR Code aparecer (tempo ajustável, ex: 30s para você escanear e digitar o código)
print("Aguardando escaneamento do QR Code e inserção do código...")
WebDriverWait(driver, 60).until(lambda d: "visao360/consult" in d.current_url)
print("QR Code validado com sucesso!")

Aguardando escaneamento do QR Code e inserção do código...
QR Code validado com sucesso!


In [ ]:
def preencher_campo_com_validacao(driver, campo_id, texto, tentativas=3):

    for tentativa in range(1, tentativas + 1):

        try:

            campo = WebDriverWait(driver, 10).until(

                EC.element_to_be_clickable((By.ID, campo_id))

            )
            
            # Limpar o campo antes de preencher
            
            campo.clear()
            time.sleep(0.3)  # Pequena pausa para garantir que o campo está limpo

            actions = ActionChains(driver)

            
            actions.click(campo).pause(0.3)
 
            for letra in texto:

                actions.send_keys(letra).pause(0.1)
 
            actions.send_keys(Keys.ARROW_DOWN).pause(0.3).send_keys(Keys.ENTER).perform()
 
            time.sleep(1.5)  # tempo para Angular aplicar a classe "ng-valid"

            classe = campo.get_attribute("class")

            if "ng-valid" in classe:

                print(f"✅ Campo '{campo_id}' preenchido e validado na tentativa {tentativa}.")

                return True

            else:

                print(f"⚠️ Tentativa {tentativa} falhou: '{campo_id}' ainda não está válido.")

        except Exception as e:

            print(f"❌ Erro na tentativa {tentativa} ao preencher o campo '{campo_id}': {e}")
 
    print(f"❌ Todas as tentativas falharam ao preencher o campo '{campo_id}'.")

    return False

 

In [8]:
def preencher_documento_com_validacao_ng_valid(driver, cpf, tentativas=3):

    for tentativa in range(1, tentativas + 1):

        try:

            campo = WebDriverWait(driver, 10).until(

                EC.element_to_be_clickable((By.NAME, "document"))

            )

            #aguardar_fim_carregamento(driver)
            
            campo.click()

            campo.clear()

            campo.send_keys(cpf)
 
            # Aguarda o Angular processar e aplicar a classe

            time.sleep(1.2)
 
            classe = campo.get_attribute("class")

            if "ng-valid" in classe:

                print(f"✅ Campo 'Documento' preenchido e validado na tentativa {tentativa}.")
                
                botao_consultar = WebDriverWait(driver, 10).until(

                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/section/sc-content/sc-consult/div/div[2]/div/sc-card-content/div/main/form/div/div[3]/sc-button/button'))

                )

                #aguardar_fim_carregamento(driver)
                
                botao_consultar.click()

                print("📤 Botão 'Consultar' clicado com sucesso.")
 
                return True

            else:

                print(f"⚠️ Tentativa {tentativa}: Documento preenchido mas ainda inválido.")

        except Exception as e:

            print(f"❌ Erro ao tentar preencher o campo 'Documento' na tentativa {tentativa}: {e}")
 
    print("❌ Todas as tentativas de validação do campo 'Documento' falharam.")

    return False

 

In [9]:
def aguardar_spinner_sumir():
    """Enquanto o spinner estiver visível, aguarda 1 segundo por ciclo, até sumir ou atingir o timeout."""
    timeout=30
    xpath_spinner = "ngx-spinner-overlay ng-tns-c59-0 ng-trigger ng-trigger-fadeIn ng-star-inserted ng-animating"
    tempo_inicial = time.time()
 
    while True:
        try:
            driver.find_element(By.CLASS_NAME, xpath_spinner)
            print("⏳ Aguardando carregamento...")
            time.sleep(1)
 
            # Interrompe se o tempo máximo for ultrapassado
            if time.time() - tempo_inicial > timeout:
                print(f"⚠️ Timeout: carregamento demorou mais que {timeout}s.")
                break
        except:
            print("✅ Spinner sumiu. Continuando o fluxo.")
            break

In [13]:
for linha, coluna in df.iterrows():
    print(linha,coluna[0],coluna[1],coluna[2],coluna[3],coluna[4])
    #cpf = l[0]
    #cpf = '05534899174'
    #cooperativa = '3179'

    #login = 'gabriel.sandres'
    #senha = 'C@tolica61136'
    
    cpf = str(coluna.iloc[0]).zfill(11)
    cooperativa = coluna[1]
    categoria = coluna[3]
    subcategoria = "Api Sicoob"
    servico = coluna[4]
    protocolo_plad = coluna[2]
    descricao = coluna[6]    
    
    
    
        # Bloco 3 – Preencher CPF/CNPJ e clicar em "Consultar"
    
    if not preencher_documento_com_validacao_ng_valid(driver,cpf):
        print("Documento inválido. Pulando para o próximo registro.")
        continue
    
    
    #-----------------------------------------------------------------------------------
    
    
    # Bloco 4 – Clicar no botão "Abrir"
    
    # Aguarda o botão "Abrir" ficar visível e clicável

    botao_abrir = WebDriverWait(driver, 10).until(

        EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/section/sc-content/sc-consult/div/div[2]/div/sc-card-content/div/main/form/div/div[4]/sc-card/div/sc-card-content/div/div/div[2]/sc-button/button'))

    )
    
    # Clica no botão "Abrir"
    #aguardar_fim_carregamento(driver)
    aguardar_spinner_sumir()
    
    botao_abrir.click()
    
    print("Botão 'Abrir' clicado com sucesso.")
    
    
    
    
    
        # Bloco 5 – Selecionar conta conforme a cooperativa
    
    # Aguarda o <select> de contas estar disponível
    select_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "accounts"))
    )
    
    # Captura todas as opções da conta
    options = select_element.find_elements(By.TAG_NAME, "option")
    
    # Percorre as opções e seleciona a que contém a cooperativa
    conta_encontrada = False
    for option in options:
        if f"Coop: {cooperativa}" in option.text:
            option.click()
            conta_encontrada = True
            print(f"Conta da cooperativa {cooperativa} selecionada com sucesso.")
            break
    
    # Se nenhuma conta compatível for encontrada
    if not conta_encontrada:
        print(f"⚠️ Nenhuma conta com cooperativa {cooperativa} encontrada.")
        
        
        
        
            # Bloco 6 – Selecionar o produto "Cobrança"
    
    # XPath do elemento que representa o card do produto "Cobrança"
    xpath_cobranca = '//*[@id="products"]/div[10]/sc-card/div/div/div/div'
    
    # Aguarda o produto "Cobrança" ficar clicável
    produto_cobranca = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath_cobranca))
    )
    
    #aguardar_fim_carregamento(driver)
    # Clica no produto "Cobrança"
    aguardar_spinner_sumir()
    produto_cobranca.click()
    time.sleep(1.5)  # tempo curto para a classe atualizar visualmente
    
    # Verifica se o produto foi selecionado (classe mudou para 'selected-product')
    pai = produto_cobranca.find_element(By.XPATH, "..")  # sobe um nível na hierarquia
    if "selected-product" in pai.get_attribute("class"):
        print("Produto 'Cobrança' selecionado com sucesso.")
    else:
        print("⚠️ Produto 'Cobrança' não foi selecionado corretamente.")
        
        
        
        
        
            # Bloco 7 – Abrir o formulário (clicar no botão com ícone de Registro de chamado)
    
    # Aguarda o botão de "Registro de chamado" estar visível e clicável
    botao_formulario = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@tooltip='Registro de chamado']"))
    )
    
    #aguardar_fim_carregamento(driver)
    # Clica no botão
    botao_formulario.click()
    aguardar_spinner_sumir()
    
    print("Formulário de chamado aberto com sucesso.")
    
    
    
    
    # Bloco 8 – Preencher corretamente o campo "Tipo de Atendimento"
    
    
    preencher_campo_com_validacao(driver, '/html/body/div[3]/div[2]/div/sc-register-ticket/sc-actionbar/div/div/div[2]/form/div/div[3]/sc-form-field/div/input', "Chat Receptivo", tentativas=3)
    
    
    
    
    
    preencher_campo_com_validacao(driver, "serviceTypeId", "Chat Receptivo")
    aguardar_spinner_sumir()
        
        
        
        
        
    # Bloco 9 – Preencher o campo "Categoria"
    
    preencher_campo_com_validacao(driver, "categoryId", categoria)
    aguardar_spinner_sumir()
        
        
        
        
    # Bloco 10 – Preencher o campo "Subcategoria"
    
    preencher_campo_com_validacao(driver, "subCategoryId", subcategoria)
    aguardar_spinner_sumir()





    # Bloco 11 – Preencher o campo "Serviço"
    
    preencher_campo_com_validacao(driver, "serviceId", servico)
    aguardar_spinner_sumir()

    
    
    
    
    # Bloco 12 – Preencher o campo "Canal de autoatendimento"
    
    # Aguarda o select ficar presente

    select_element = WebDriverWait(driver, 10).until(

        EC.presence_of_element_located((By.ID, "Canal De Autoatendimento"))

    )
    
    # Usa Select do Selenium para selecionar pelo texto visível

    select = Select(select_element)

    select.select_by_visible_text("Não Se Aplica")
    
    # Aguarda e valida se o campo foi aceito (classe ng-valid)

    time.sleep(0.5)

    classe_atual = select_element.get_attribute("class")

    if "ng-valid" in classe_atual:

        print("✅ Campo 'Canal de autoatendimento' selecionado com sucesso.")

    else:

        print("⚠️ O campo 'Canal de autoatendimento' foi preenchido, mas não foi validado pela aplicação.")

    
    
    
    
    
    # Bloco 13 – Preencher o campo "Protocolo PLAD"
    
    # Aguarda o campo estar clicável

    campo_protocolo = WebDriverWait(driver, 10).until(

        EC.element_to_be_clickable((By.ID, "Protocolo Plad"))

    )
    
    # Clica e digita o valor da variável protocolo_plad
    #aguardar_fim_carregamento(driver)
    
    campo_protocolo.click()

    campo_protocolo.clear()

    campo_protocolo.send_keys(protocolo_plad)
    
    # Aguarda e valida o preenchimento

    time.sleep(1)

    classe_atual = campo_protocolo.get_attribute("class")

    if "ng-valid" in classe_atual:

        print("✅ Campo 'Protocolo PLAD' preenchido e validado com sucesso.")

    else:

        print("⚠️ O campo 'Protocolo PLAD' foi preenchido, mas não foi validado pela aplicação.")





    # Bloco 14 – Preencher o campo "Descrição"
    
    # Verifica se a descrição é válida ou precisa usar fallback

    # Trata o NaN e verifica se tem conteúdo válido
    
    if pd.isna(descricao) or not isinstance(descricao, str) or len(descricao.strip()) < 10:
        descricao_final = "Chamado registrado automaticamente via RPA"
    else:
        descricao_final = descricao.strip()

        
    # Aguarda o textarea de descrição ficar clicável

    campo_descricao = WebDriverWait(driver, 10).until(

    EC.element_to_be_clickable((By.ID, "description"))

        )
        
    # Clica, limpa e digita a descrição

    #aguardar_fim_carregamento(driver)
    
    campo_descricao.click()

    campo_descricao.clear()

    campo_descricao.send_keys(descricao_final)
        
        # Aguarda e valida o preenchimento

    time.sleep(1)

    classe_atual = campo_descricao.get_attribute("class")

    if "ng-valid" in classe_atual:

        print("✅ Campo 'Descrição' preenchido e validado com sucesso.")

    else:

        print("⚠️ O campo 'Descrição' foi preenchido, mas não foi validado pela aplicação.")
    
    
    
    
    # Bloco 15 – Clicar no botão "Registrar"
    
    # Aguarda o botão "Registrar" ficar clicável

    botao_registrar = WebDriverWait(driver, 20).until(

        EC.element_to_be_clickable((By.XPATH, '//*[@id="actionbar hide"]/div/div[2]/form/div/div[20]/sc-button/button'))

    )
    
    # Clica no botão
    #aguardar_fim_carregamento(driver)
    
    botao_registrar.click()
    
    print("✅ Primeiro clique no botão 'Registrar' realizado com sucesso.")




    # Bloco 16 – Clicar no segundo botão "Registrar" (confirmação do modal)
    
    # Aguarda o botão "Registrar" dentro do modal ficar clicável

    botao_confirmar_registro = WebDriverWait(driver, 20).until(

        EC.element_to_be_clickable((By.XPATH, '//*[@id="modal"]/div/sc-modal-footer/div/div/div[2]/sc-button/button'))

    )
    
    # Clica no botão de confirmação
    #aguardar_fim_carregamento(driver)
    
    botao_confirmar_registro.click()
    
    print("✅ Segundo clique no botão 'Registrar' (confirmação) realizado com sucesso.")




    # Aguarda o elemento com o protocolo ficar visível
    elemento_protocolo = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="actionbar hide"]/div/div[2]/form/div/div[2]/sc-card/div/sc-card-content/div/div/div[1]/h5'))
    )

    # Extrai o texto (número do protocolo)
    numero_protocolo = elemento_protocolo.text.strip()
    print(f"✅ Protocolo capturado: {numero_protocolo}")


    # Atualiza o valor na linha correspondente (índice i dentro do loop for)
    df.loc[linha, 'Protocolo Visão'] = numero_protocolo

    # Salva de volta
    df.to_excel(folderFile, index=False)
    print("✅ Protocolo colado na planilha com sucesso.")




    # Bloco – Clicar em "Finalizar Atendimento"

    try:

        # Aguarda o botão "Finalizar Atendimento" estar clicável

        btn_finalizar = WebDriverWait(driver, 20).until(

            EC.element_to_be_clickable((By.XPATH, '//*[@id="actionbar hide"]/div/div[2]/form/div/div[5]/sc-button/button'))

        )
        #aguardar_fim_carregamento(driver)
        
        btn_finalizar.click()

        print("✅ Botão 'Finalizar Atendimento' clicado com sucesso.")

    except Exception as e:

        print("❌ Erro ao clicar no botão 'Finalizar Atendimento':", e)

    
    
    
    
    
        # Bloco – Clicar em "Confirmar" para finalizar o atendimento

        # Aguarda o botão 'Confirmar' estar clicável E sem overlay por cima

    try:

        WebDriverWait(driver, 10).until(

            EC.element_to_be_clickable((By.XPATH, '//*[@id="modal"]/div/main/div/div[4]/button'))

        )
    
        # Usa JavaScript para clicar se houver interceptação visual
        #aguardar_fim_carregamento(driver)
        
        botao_confirmar = driver.find_element(By.XPATH, '//*[@id="modal"]/div/main/div/div[4]/button')

        driver.execute_script("arguments[0].click();", botao_confirmar)
    
        print("✅ Botão 'Confirmar' clicado com sucesso (via JS).")
    
    except Exception as e:

        print("❌ Erro ao clicar no botão 'Confirmar':", e)

 
 
 
    
    print("Fim  loop")
 
    

C:\Users\gabriel.sandres\AppData\Local\Temp\ipykernel_25280\2970224031.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(linha,coluna[0],coluna[1],coluna[2],coluna[3],coluna[4])
C:\Users\gabriel.sandres\AppData\Local\Temp\ipykernel_25280\2970224031.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cooperativa = coluna[1]
C:\Users\gabriel.sandres\AppData\Local\Temp\ipykernel_25280\2970224031.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, 

0 05534899174 3179 WP12345678 Pix Dúvida Técnica
✅ Campo 'Documento' preenchido e validado na tentativa 1.
📤 Botão 'Consultar' clicado com sucesso.
✅ Spinner sumiu. Continuando o fluxo.
Botão 'Abrir' clicado com sucesso.
Conta da cooperativa 3179 selecionada com sucesso.
✅ Spinner sumiu. Continuando o fluxo.
⚠️ Produto 'Cobrança' não foi selecionado corretamente.
✅ Spinner sumiu. Continuando o fluxo.
Formulário de chamado aberto com sucesso.
❌ Erro na tentativa 1 ao preencher o campo '/html/body/div[3]/div[2]/div/sc-register-ticket/sc-actionbar/div/div/div[2]/form/div/div[3]/sc-form-field/div/input': Message: 
Stacktrace:
	GetHandleVerifier [0x0x7f3783+63299]
	GetHandleVerifier [0x0x7f37c4+63364]
	(No symbol) [0x0x621113]
	(No symbol) [0x0x66987e]
	(No symbol) [0x0x669c1b]
	(No symbol) [0x0x6b2212]
	(No symbol) [0x0x68e5c4]
	(No symbol) [0x0x6afa4a]
	(No symbol) [0x0x68e376]
	(No symbol) [0x0x65d6e0]
	(No symbol) [0x0x65e544]
	GetHandleVerifier [0x0xa4e073+2531379]
	GetHandleVerifier [

KeyboardInterrupt: 

In [ ]:
def aguardar_spinner_sumir(xpath_spinner_var):
    """Enquanto o spinner estiver visível, aguarda 1 segundo por ciclo, até sumir ou atingir o timeout."""
    timeout=30
    xpath_spinner = xpath_spinner_var
    tempo_inicial = time.time()
 
    while True:
        try:
            driver.find_element(By.CLASS_NAME, xpath_spinner)
            print("⏳ Aguardando carregamento...")
            time.sleep(1)
 
            # Interrompe se o tempo máximo for ultrapassado
            if time.time() - tempo_inicial > timeout:
                print(f"⚠️ Timeout: carregamento demorou mais que {timeout}s.")
                break
        except:
            print("✅ Spinner sumiu. Continuando o fluxo.")
            break

In [ ]:
print("Todos chamados registrados")

In [ ]:
aguardar_spinner_sumir("ngx-spinner-overlay ng-tns-c59-0 ng-trigger ng-trigger-fadeIn ng-star-inserted ng-animating")